In [1]:
import csv
import torch
import torchvision
import string
import matplotlib.pyplot as plt
import random
import json
import time
def outerwordToTensor(sentence,all_letters): # use
        tensor = torch.zeros([len(sentence),1,n_letters],dtype=torch.int32)
        for word in range(len(sentence)):
            tensor[word][0][all_letters.find(sentence[word])] = 1
        return tensor.float()


class star_dataset(torch.utils.data.Dataset):
    def __init__(self, data,all_letters):
        self.relevant_data = data
        self.all_letters = all_letters
    def wordToTensor(self,sentence): # use
        tensor = torch.zeros([len(sentence),1,n_letters],dtype=torch.int32)
        for word in range(len(sentence)):
            tensor[word][0][self.all_letters.find(sentence[word])] = 1
        return tensor.float()
    def __getitem__(self,index):
#         print(self.relevant_data[index])
        return self.wordToTensor(self.relevant_data[index])
    def __len__(self):
        return len(self.relevant_data)
def do_data():
    return_data = []
    filterwords = ["NEXTEPISODE"]
    all_words = set()
    reader = csv.reader(open("datasets/star_trek_transcripts_all_episodes.csv",newline = ""),delimiter = ",",quotechar ='"')
    for row in reader:
        for sentence in row:
            if sentence not in filterwords and len(sentence)>1:
                working = sentence.strip().replace(";","").replace('\"',"").replace("=","").replace("/"," ").replace('+','').replace('-',"").replace("'","").replace("[","").replace("]","").replace("*","")
                return_data.extend([working+"*"])
                for i in sentence.split():
                    all_words.add(i) # in case.. i suppose
    return return_data, all_words
    

working_data,all_words= do_data()
n_samples = len(working_data)
all_letters = string.ascii_letters + "0123456789 .,:!?()*" #* is the EOS token.
n_letters = len(all_letters)
print(working_data[0]) # check that this returns a sentence
print(list(all_words)[0]) #check this returns a complete word
# print(outerwordToTensor("pp oo",all_letters).shape)
# print(outerwordToTensor("pp oo",all_letters))

SPOCK: Check the circuit.*
Evacuate


In [2]:
class lstm(torch.nn.Module):
    def __init__(self, input_size, hidden_dim,device,num_layers = 2, tempy =None):
        super(lstm,self).__init__()
        self.device = device
        self.lstm = torch.nn.LSTM(input_size =input_size,hidden_size =hidden_dim ,num_layers = num_layers,batch_first =True, dropout=0.1)
        self.linear = torch.nn.Linear(hidden_dim,input_size)
        self.hidden = hidden_dim
        self.tempy = tempy
        self.num_layers = num_layers
        self.softmax = torch.nn.LogSoftmax(dim =1)
    def forward(self, input , hidden,do_temp):
        out1,hidden = self.lstm(input,hidden)
#         print (out1.shape)
        out2 = self.linear(out1)
#         print(out2.shape)
        if self.tempy and do_temp:
            out2/=self.tempy
        output = self.softmax(out2.squeeze(0).squeeze(1))### THIS IS A WEIRD THING
        # also, current nll reads as batch size..?
        return output, hidden
    def initHidden(self):
        return ([torch.zeros(self.num_layers,1, self.hidden).to(self.device),torch.zeros(self.num_layers,1, self.hidden).to(self.device)])

In [3]:
hidden_dim = 200
total_epoch = 30
learning_rate = 0.001
momentum_mod = 0.9
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
my_model = lstm(n_letters,hidden_dim,device,n_letters,tempy=0.5)
my_model = my_model.to(device)
loss_function = torch.nn.NLLLoss()
optimizer = torch.optim.SGD(my_model.parameters(), lr=learning_rate,momentum = momentum_mod)
torch.optim.lr_scheduler.StepLR(optimizer,step_size = 1,gamma = 0.1,last_epoch =-1)
random.shuffle(working_data)
train = working_data[:int(len(working_data)/10*7)]
test = working_data[int(len(working_data)/10*7):]
train_loader = torch.utils.data.DataLoader(star_dataset(train,all_letters),batch_size =1,shuffle = True,num_workers=1)
test_loader = torch.utils.data.DataLoader(star_dataset(test,all_letters),batch_size =1,shuffle = True,num_workers=1)
total_loss_list = []
generateds = {}
test_loss=[]
accuracies = []

In [31]:
def generate(start_letter='K',max_length =30):
    with torch.no_grad():  # no need to track history in sampling
        input =outerwordToTensor(start_letter,all_letters)
        output_phrase = start_letter
        hidden = my_model.initHidden()
        for i in range(max_length):
            output, hidden = my_model(input.to(device),hidden,True)
#             topv, topi = output.topk(1)
#             topi = topi[0][0]
            topv, topi = output.topk(1)
            for sampled_letter in output:
                sampled = int(torch.distributions.categorical.Categorical(torch.exp(sampled_letter)).sample())
#                 sampled = topi
                letter = all_letters[sampled]
                output_phrase += letter
            if letter=="*": # is end of sentence
                break
            input = outerwordToTensor(letter,all_letters)
        return output_phrase[:-1]
my_model.load_state_dict(torch.load(f"startrekmodels/startrekgen_{0}.pt")) # load the previous things.
print(generate("K"))
print(len(train_loader))
print(len(test_loader))

KKAOO I      nuocooe oe ee ntr
25437
10902


In [32]:
for i in range(100):
    print(generate())

KHKRT::   ae   egt  sa i s ua 
KKRIT::: raan    s  sbtaclo t 
KCWKI::   e  na e
KPTK::   aw ti nts ehr    i hn
KKKOI:: s  eantye  eecsem  deo
KRRKK:: o I  ae ti e n e   oii
KRKIN: : ieo u    nn ao   nt  
KIKRK:   t  iea oeunt 
KCO:A:Y  eee  s .aw
KSOIR   te o e edo  to g d l  
KAKROR  eYyarsoeat seetr e sne
KRKR:O:  t oiate te  t  hg 
KKKKO::: ue  a   nh  ettt  nee
KRKOI::I  anan    iati t cee  
KIKOK:    r  oiatni smylniee  
KKKIO:      t a h  eens ea .  
KSC:O:   aerts  taa s odis ii 
K8AEO::  t, ieden tesgeah  eus
KRKKR::   irtroco     e. ep
KKKOO:   ia cscii  e  oe     e
KKI:R:   bie eaaucto i   gn   
KRKU::  Aoruo e    e.n    
KCKRR::: he  rr cndist   dle o
KKKCO:::  haleoha loe ade  te 
KKRTK::: oeate eo  r  e   e a
KwRKE:::Y a e oeu   o.ldeoes r
KpKR:::o so s soee   otr   ieo
KKKKY::  reaeuei  u  i t  .ro 
KOCRK  :s   ealna     ons o. d
KSKRA :L   ot r  o ituy ete  
KSEK C:    ts aoo   e   d eia 
KIRK : :  t  to l   a otkntt t
KKCKRI  e a  hnan r.t en he re
KPRO:R:: When et, ea 

In [44]:
current_epoch = 0
# if current_epoch!=0:
#     my_model.load_state_dict(torch.load(f"startrekgen_{current_epoch-1}.pt")) # load the previous things.
#     generateds = json.load(open("output_generations.json","r"))

for epoch_num in range(current_epoch,total_epoch):
    print("beginning a new epoch")
    total_loss =0
    totes_counter=0
    generateds[epoch_num] = []
    batch_time = time.time()
    for batch_number, values in enumerate(train_loader):
        my_model.train()
        my_model.zero_grad()
        loss =0
        latest_hidden =my_model.initHidden()
        values = values.squeeze(0)
        answer = torch.nonzero(values.squeeze(1))[:,1]
#         print(answer.shape)
#         print(values.shape)
        values = values.to(device)
        answer = answer.to(device)
        output,hidden = my_model(values.view(values.shape[1],values.shape[0],values.shape[2]),latest_hidden,False)
#         print(output.shape)
        l = loss_function(output,answer.long())
        total_loss+=l.item()
        loss+=l
        l.backward()
        optimizer.step()
        latest_timing = time.time()- batch_time
        batch_time = time.time()
        print(f"batch done.{latest_timing}")
        
        if batch_number%2000==0 and batch_number!=0:
            temporarylist = [] # Generate a batch of 20
            my_model.eval()
            for genny in range(20):
                temporarylist.append(generate(all_letters[random.randint(26,51)]))
                generateds[epoch_num].append(temporarylist)
            print("Generated 20 samples..")
            print(temporarylist)
            batch_time = time.time() # reset the timing..
            
            
        if batch_number%int(len(train_loader)/10)==0 and batch_number!= 0 :
            totes_counter+=1
            print(f"{totes_counter*10}% done")
    
#     losshold.backward()
#     optimizer.step()
#     optimizer.zero_grad()
#     losshold = None
    temporarylist = []
    for genny in range(20):
        temporarylist.append(generate(all_letters[random.randint(26,51)]))
        generateds[epoch_num].append(temporarylist)
    json.dump(generateds,open("startrekmodels/output_generations.json","w"))
    print(f"epoch complete. Total loss = {total_loss}")
    total_loss_list.append(total_loss)
    torch.save(my_model.state_dict(),f"startrekmodels/startrekgen_{epoch_num}.pt")
    correct =0   
    total_loss = 0
    total =0
    my_model.eval()
    print("Beginning Evaluation.")
    for batch_number, values in enumerate(test_loader):
        my_model.train()
        my_model.zero_grad()
        loss =0
        latest_hidden =my_model.initHidden()
        values = values.squeeze(0)
        answer = torch.nonzero(values.squeeze(1))[:,1]
#         print(answer.shape)
#         print(values.shape)
        values = values.to(device)
        answer = answer.to(device)
        output,hidden = my_model(values.view(values.shape[1],values.shape[0],values.shape[2]),latest_hidden,False)
#         print(output.shape)
        values = torch.nonzero(values.squeeze(0).squeeze(0))[:,1]
        l = loss_function(output,values.long())
        total_loss+=l.item()
        topv, topi = output.topk(1)
        topi = topi.to("cpu")
        values = values.to("cpu")
        for i in range(len(topi)):
            total+=1
            if topi[i]==values[i]:
                correct+=1
        if batch_number%1000 ==0 and batch_number!=0:
            print("completed 1k of test batch")
    print(f"Loss = {total_loss}")
    print(f"Accuracies = {correct/total}")
    test_loss.append(total_loss)
    accuracies.append(correct/total)
    json.dump(test_loss,open("startrekmodels/test_loss.json","w"))
    print("Test losses dumped.")
    json.dump(accuracies,open("startrekmodels/accuracies.json","w"))
    print("Test Accuracies dumped.")

beginning a new epoch
batch done.1.3741154670715332
batch done.0.5411343574523926
batch done.1.3159565925598145


Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

In [43]:
test_loss=[]
accuracies = []
total = len(test_loader)
total =0
for batch_number, values in enumerate(test_loader):
    my_model.train()
    my_model.zero_grad()
    loss =0
    latest_hidden =my_model.initHidden()
    values = values.squeeze(0)
    print(values.shape)
    answer = torch.nonzero(values.squeeze(1))[:,1]
    print(answer.shape)
    print(answer)
    print(values.shape)
    values = values.to(device)
    answer = answer.to(device)
    output,hidden = my_model(values.view(values.shape[1],values.shape[0],values.shape[2]),latest_hidden,False)
#         print(output.shape)
    values = torch.nonzero(values.squeeze(1))[:,1]
    l = loss_function(output,answer.to(device))
    total_loss+=l.item()
    topv, topi = output.topk(1)
    topi = topi.to("cpu")
    values = values.to("cpu")
    answer = answer.to("cpu")
    for i in range(len(topi)):
        total+=1
        if topi[i]==answer[i]:
            correct+=1
    if batch_number%1000 ==0 and batch_number!=0:
        print("completed 1k of test batch")
    print(answer)
    print(output)
    print(values)
    break
    
test_loss.append(total_loss)
accuracies.append(correct/total)
print(correct)
print(total)
print(test_loss)

torch.Size([64, 1, 71])
torch.Size([64])
tensor([36, 34, 43, 36, 65, 62, 37,  8,  4, 20, 19,  4, 13,  0, 13, 19, 62, 44,
        20, 11, 20, 64, 62,  3, 14, 62, 24, 14, 20, 62,  5,  4,  4, 11, 62, 18,
        19, 17, 14, 13,  6, 62,  4, 13, 14, 20,  6,  7, 62, 19, 14, 62, 12, 14,
        21,  4, 62,  0,  1, 14, 20, 19, 67, 70])
torch.Size([64, 1, 71])
tensor([36, 34, 43, 36, 65, 62, 37,  8,  4, 20, 19,  4, 13,  0, 13, 19, 62, 44,
        20, 11, 20, 64, 62,  3, 14, 62, 24, 14, 20, 62,  5,  4,  4, 11, 62, 18,
        19, 17, 14, 13,  6, 62,  4, 13, 14, 20,  6,  7, 62, 19, 14, 62, 12, 14,
        21,  4, 62,  0,  1, 14, 20, 19, 67, 70])
tensor([[-5.0166e-02, -7.3258e+00, -7.3415e+00,  ..., -7.3051e+00,
         -7.2219e+00, -7.2726e+00],
        [-1.5097e-03, -1.0838e+01, -1.0916e+01,  ..., -1.0771e+01,
         -1.0661e+01, -1.0710e+01],
        [-9.7275e-05, -1.3605e+01, -1.3727e+01,  ..., -1.3499e+01,
         -1.3367e+01, -1.3422e+01],
        ...,
        [ 0.0000e+00, -1.9204e+01, 

In [ ]:
# total =0
# my_model.eval()
# for batch_number, values in enumerate(test_loader):
#     output,hidden = my_model(values.squeeze(0).to(device),my_model.initHidden(),False)
#     values = torch.nonzero(values.squeeze(0).squeeze(0))[:,1]
#     l = loss_function(output,values.long().to(device))
#     total_loss+=l.item()
#     topv, topi = output.topk(1)
#     topi = topi.to("cpu")
#     values = values.to("cpu")
#     for i in range(len(topi)):
#         total+=1
#         if topi[i]==values[i]:
#             correct+=1
    

In [ ]:
# test_loss.append(total_loss)
# accuracies.append(correct/total)
# json.dump(test_loss,open("test_loss.json","w"))
# json.dump(accuracies,open("accuracies.json","w"))